In [1]:
from django.templatetags.i18n import language
from docutils.nodes import topic
from flask.cli import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

from langchain.prompts import (
    ChatMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from numpy.ma.core import product
from sphinx.cmd.quickstart import suffix

In [7]:
info = "Name: {0}, Age: {1}".format("Jerry", 25)
print(info)

info = "Name: {name}, Age: {age}".format(name="Tom", age=25)
print(info)

person = {"name": "David", "age": 40}
info = "Name: {name}, Age: {age}".format(**person)
print(info)

Name: Jerry, Age: 25
Name: Tom, Age: 25
David 40
Name: David, Age: 40


# 具体使用：PromptTemplate
## 方式1：使用构造方法

In [9]:
from langchain.prompts import PromptTemplate

# 自定义模板：描述主题的应用
template = PromptTemplate(
    template="请简要描述{topic}的应用",
    input_variables=["topic"]
)
print(template)

# 使用模板生成提示词
prompt_1 = template.format(topic="机器学习")
prompt_2 = template.format(topic="自然语言处理")

print("提示词1:", prompt_1)
print("提示词2:", prompt_2)

input_variables=['topic'] input_types={} partial_variables={} template='请简要描述{topic}的应用'
提示词1: 请简要描述机器学习的应用
提示词2: 请简要描述自然语言处理的应用


In [11]:
from langchain.prompts import PromptTemplate

# 定义多变量模板
template = PromptTemplate(
    template="请评价{product}的优缺点，包括{aspect1}和{aspect2}。",
    input_variables=["product", "aspect1", "aspect2"]
)

# 使用模板生成提示词
prompt_1 = template.format(product="智能手机", aspect1="电池续航", aspect2="拍照质量")
prompt_2 = template.format(product="MacBook Pro", aspect1="屏幕尺寸", aspect2="价格")

print("提示词1:", prompt_1)
print("提示词2:", prompt_2)

提示词1: 请评价智能手机的优缺点，包括电池续航和拍照质量。
提示词2: 请评价MacBook Pro的优缺点，包括屏幕尺寸和价格。


In [13]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "请给我一个关于{topic}的{type}解释"
)

prompt = prompt_template.format(topic="量子力学", type="详情")
print(prompt)

请给我一个关于量子力学的详情解释


In [14]:
# 1.导入相关的包
from langchain_core.prompts import PromptTemplate

# 2. 定义提示词模板对象
text = """
Tell me a joke
"""

prompt_template = PromptTemplate.from_template(text)
# 3. 默认使用f-string进行格式化（返回格式化好的字符串）
prompt = prompt_template.format()
print(prompt)


Tell me a joke



# 两种新的结构形式
## 形式1：部分提示词模版

In [16]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(
    template="{foo}{bar}",
    input_variables=["foo", "bar"],
    partial_variables={"foo": "hello"}
)

prompt = template.format(bar="world")
print(prompt)

helloworld


In [17]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(
    template="{foo}{bar}",
    input_variables=["foo", "bar"]
)

partial_template = template.partial(foo="hello")

prompt = partial_template.format(bar="world")
print(prompt)

helloworld


In [1]:
from langchain_core.prompts import PromptTemplate

# 完整模板
full_template = """
你是一个{role}，请用{style}风格回答：
问题：{question}
答案：
"""

# 预填充角色和风格
partial_template = PromptTemplate.from_template(full_template).partial(
    role="资深厨师",
    style="专业但幽默"
)

# 只需要提供剩余变量
print(partial_template.format(question="如何煎牛排？"))


你是一个资深厨师，请用作业但幽默风格回答：
问题：如何煎牛排？
答案：



In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    template="请评价{product}的优缺点，包括{aspect1}和{aspect2}。",
    partial_variables={"aspect1": "电池", "aspect2": "屏幕"}
)

prompt = prompt_template.format(product="笔记本电脑")
print(prompt)

请评价笔记本电脑的优缺点，包括电池和屏幕。


In [4]:
from langchain_core.prompts import PromptTemplate

template = (
        PromptTemplate.from_template("Tell me a joke about {topic}")
        + ", make it funny"
        + "\n\nand in {language}"
)

prompt = template.format(topic="sports", language="spanish")
print(prompt)

Tell me a joke about sports, make it funny

and in spanish


# format() 与 invoke()

In [5]:
from langchain_core.prompts import PromptTemplate

# 定义提示词模板对象
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# 默认使用f-string格式化
prompt_template.invoke({"adjective": "funny", "content": "chickens"})

StringPromptValue(text='Tell me a funny joke about chickens.')

In [6]:
from langchain_core.prompts import PromptTemplate

# 使用初始化进行实例化
prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}"
)

# PromptTemplate底层是RunnableSerializable接口 所以可以直接使用invoke()调用
prompt.invoke({"adjective": "funny", "content": "chickens"})

StringPromptValue(text='Tell me a funny joke about chickens')

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = (
        PromptTemplate.from_template("Tell me a joke about {topic}")
        + ", make it funny"
        + " and in {language}"
)

prompt = prompt_template.invoke({"topic": "sports", "language": "spanish"})
print(prompt)

text='Tell me a joke about sports, make it funny and in spanish'


In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

llm = ChatOpenAI(model=os.getenv("LLM_MODEL"))

prompt_template = PromptTemplate.from_template(
    template="请评价{product}的优缺点，包括{aspect1}和{aspect2}。"
)

product = input("请输入产品名称：")
aspect1 = input("请输入第一个评价项：")
aspect2 = input("请输入第二个评价项：")
prompt = prompt_template.format(product=product, aspect1=aspect1, aspect2=aspect2)

print(type(prompt))

# llm.invoke(prompt)
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)


<class 'str'>

好的，这是一个对现代智能手机的全面评价，涵盖了其通用优缺点，并重点分析了AI赋能和AI语音助手的特定方面。

---

### **对现代智能手机的综合评价**

智能手机已经从简单的通讯工具，演变成了我们生活中不可或缺的“数字中枢”。它集通讯、信息、娱乐、工作、支付等功能于一身，而近年来人工智能（AI）的深度赋能，更是极大地提升了其能力边界，但同时也带来了新的挑战。

---

### **一、 手机的总体优缺点**

#### **优点**

1.  **极致的便捷性与连接性：** 随时随地与世界保持连接。通过电话、短信、社交媒体、即时通讯工具，人与人之间的沟通变得前所未有的简单高效。
2.  **强大的信息获取能力：** 整个互联网尽在指尖。无论是学习新知识、查询资料、阅读新闻，还是观看教学视频，手机都是最直接的入口。
3.  **高效的生产力工具：** 邮件、日历、云文档、视频会议等应用让移动办公成为可能。你可以随时随地处理工作，大大提高了灵活性和效率。
4.  **丰富的娱乐体验：** 高清屏幕、立体声扬声器、强大的图形处理能力，使其成为绝佳的游戏机、电影院和音乐播放器。
5.  **全能的生活助手：** 移动支付、地图导航、在线购物、外卖点餐、打车出行……手机几乎渗透到现代生活的每一个角落，极大地简化了日常事务。
6.  **出色的影像记录能力：** 多摄像头系统、计算摄影技术的进步，让普通人也能轻松拍出高质量的照片和视频，记录生活中的精彩瞬间。

#### **缺点**

1.  **健康隐患：**
    *   **视力问题：** 长时间盯着小屏幕容易导致眼干、眼疲劳和近视加深。
    *   **颈椎问题：** 低头姿势是颈椎病的诱因之一。
    *   **睡眠干扰：** 屏幕发出的蓝光会抑制褪黑素分泌，影响睡眠质量。
    *   **心理问题：** 信息过载、社交攀比可能导致焦虑和抑郁。
2.  **信息过载与成瘾性：** 无休止的通知推送、算法推荐的信息流，容易让人陷入“数字沉迷”，消耗大量时间精力，难以专注。
3.  **隐私与安全风险：** 手机存储了大量个人敏感信息（照片、联系人、位置、支付信息等），一旦被黑客攻击或数据被滥用，后果不堪设想。App过度索权也是普遍问题。
4.  **高昂的成本：*

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

llm = ChatOpenAI(model=os.getenv("LLM_MODEL"))

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])
# prompt_template.format_messages(msgs=[HumanMessage(content="hi!")])

message = prompt_template.format_messages(msgs=[HumanMessage(content="hi!")])
# for chunk in llm.stream(message):
#     print(chunk.content, end="", flush=True)
print(message)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

llm = ChatOpenAI(model=os.getenv("LLM_MODEL"))

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ]
)

msg = prompt.format_messages(
    history=[HumanMessage(content="1+2*3=?"), AIMessage(content="1+2*3=7")],
    question="我刚才的问题是什么？"
)
print(msg)
print("------")
print(llm.invoke(msg))

[SystemMessage(content='You are helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='1+2*3=?', additional_kwargs={}, response_metadata={}), AIMessage(content='1+2*3=7', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚才的问题是什么？', additional_kwargs={}, response_metadata={})]
------
content='\n您刚才的问题是：1+2*3=?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 333, 'prompt_tokens': 37, 'total_tokens': 370, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 6}}, 'model_name': 'glm-4.6', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--65ba9739-b647-4caf-b0dd-6eeaa7fb9463-0' usage_metadata={'input_tokens': 37, 'output_tokens': 333, 'total_tokens': 370, 'input_token_details': {'cache_read': 6}, 'output_token_details': {}}


In [24]:
from langchain_core.prompts import (ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,
                                    SystemMessagePromptTemplate)
from langchain_core.messages import SystemMessage

# 定义消息模板
prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template("你是{role}"),
        MessagesPlaceholder(variable_name="intermediate_steps"),
        HumanMessagePromptTemplate.from_template("{query}")
    ]
)

# 定义消息对象
intermediate = [
    SystemMessage(name="search", content="厦门: 晴, 25℃")
]

# 格式化聊天消息提示词模板
prompt.format_messages(
    role="天气预报员",
    intermediate_steps=intermediate,
    query="厦门天气怎么样？"
)

[SystemMessage(content='你是天气预报员', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='厦门: 晴, 25℃', additional_kwargs={}, response_metadata={}, name='search'),
 HumanMessage(content='厦门天气怎么样？', additional_kwargs={}, response_metadata={})]

In [26]:
from langchain_core.prompts import (ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder)
from langchain_core.messages import AlMessage, HumanMessage

# 定义HumanMessage对象
human_message = HumanMessage(content="学习编程的最好方法是什么？")

# 定义AIMessage对象
ai_message = AIMessage(
    content="""|
    1.选择一门编程语言：选择一门你想学习的编程语言
    2.从基础开始：熟悉基本的编程概念，如变量、数据类型和控制结构
    3.练习，练习，再练习：学习编程的最好方法是通过实践经验
    |
    """
)

# 定义提示词
human_prompt = "用{word_count}个词总结我们到目前为止的对话"

# 定义提示词模板
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"),
    human_message_template
])

# 格式化聊天消息提示词模板
message = chat_prompt.format_messages(
    conversation=[human_message, ai_message], word_count="10"
)
print(message)

[HumanMessage(content='学习编程的最好方法是什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='|\n    1.选择一门编程语言：选择一门你想学习的编程语言\n    2.从基础开始：熟悉基本的编程概念，如变量、数据类型和控制结构\n    3.练习，练习，再练习：学习编程的最好方法是通过实践经验\n    |\n    ', additional_kwargs={}, response_metadata={}), HumanMessage(content='用10个词总结我们到目前为止的对话', additional_kwargs={}, response_metadata={})]


In [29]:
from langchain_openai import ChatOpenAI
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL"), temperature=0.4)

res = chat_model.invoke("2 🦁 17 = ?")
print(res.content)


The answer is **August**.

Here's the logic:

1.  The 🦁 (lion) represents the zodiac sign **Leo**.
2.  The numbers **2** and **17** represent a range of days in a month (the 2nd to the 17th).
3.  The zodiac sign for Leo is from **July 23 to August 22**.
4.  The question is asking: In which month does the 2nd to the 17th fall entirely within the Leo dates?
5.  In **August**, the period from August 2nd to August 17th is completely within the Leo timeframe.


In [31]:
from langchain.prompts.few_shot import FewShotPromptTemplate

# 创建示例集合
examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "武汉热吗", "output": "武汉市"}
]

# 创建PromptTemplate示例
example_prompt = PromptTemplate.from_template(
    template="input: {input}\nOutput:{output}"
)

# 创建FewShotPromptTemplate实例
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="input: {input}\nOutput:",  # 要放在示例后面的提示模板字符串
    input_variables=["input"]  # 传入的变量
)

# 调用
prompt = prompt.invoke({"input": "长沙多少度？"})
print(prompt)

text='input: 北京天气怎么样\nOutput:北京市\n\ninput: 南京下雨吗\nOutput:南京市\n\ninput: 武汉热吗\nOutput:武汉市\n\ninput: 长沙多少度？\nOutput:'


In [5]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

dotenv.load_dotenv()

llm = ChatOpenAI(model=os.getenv("LLM_MODEL"), temperature=0.4)

# 创建示例集合
examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "武汉热吗", "output": "武汉市"}
]

# 创建PromptTemplate示例
example_prompt = PromptTemplate.from_template(
    template="input: {input}\nOutput:{output}"
)

# 创建FewShotPromptTemplate实例
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="input: {input}\nOutput:",  # 要放在示例后面的提示模板字符串
    input_variables=["input"]  # 传入的变量
)

# 调用
prompt = prompt.invoke({"input": "长沙多少度？"})
res = llm.invoke(prompt)
print(res.content)


长沙市


In [1]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# 创建提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数字专家，算式：{input} 值：{output} 使用：{description}"

# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)

# 提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]

# 创建一个FewShotPromptTemplate对象
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学家，算式：{input} 值：{output}",
    input_variables=["input", "output"]
)

print(prompt.invoke({"input": "2*5", "output": "10"}))
print("*" * 20)

# 初始化大模型，然后调用
dotenv.load_dotenv()

llm = ChatOpenAI(model=os.getenv("LLM_MODEL"))

result = llm.invoke(prompt.invoke({"input": "2*5", "output": "10"}))
print(result.content)

text='你是一个数字专家，算式：2+2 值：4 使用：加法运算\n\n你是一个数字专家，算式：5-2 值：3 使用：减法运算\n\n你是一个数学家，算式：2*5 值：10'
********************

使用：乘法运算


In [4]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

# 示例消息格式
examples = [
    {"input": "1+1等于几？", "output": "1+1=等于2"},
    {"input": "法国的首都是？", "output": "巴黎"}
]

# 定义示例的消息格式提示词模板
msg_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_example_prompt,
    examples=examples
)

# 输出格式化后的消息
print(few_shot_prompt.format())

Human: 1+1等于几？
AI: 1+1=等于2
Human: 法国的首都是？
AI: 巴黎


In [2]:
# 导入相关包
import os
import dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()

# 定义嵌入模型
embeddings_model = OpenAIEmbeddings(
    model=os.getenv("LLM_EMBEDINGS_MODEL")
)

# 定义示例组
examples = [
    {
        "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵?",
        "answer": """
接下来还需要问什么问题吗？
追问：穆罕默德·阿里去世时多大年纪？
中间答案：穆罕默德·阿里去世时享年74岁。
""",
    },
    {
        "question": "craigslist的创始人是什么时候出生的？",
        "answer": """
接下来还需要问什么问题吗？
追问：谁是craigslist的创始人？
中级答案：Craigslist是由克雷格·纽马克创立的。
""",
    },
    {
        "question": "谁是乔治·华盛顿的外祖父？",
        "answer": """
接下来还需要问什么问题吗？
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
""",
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
        "answer": """
接下来还需要问什么问题吗？
追问：《大白鲨》的导演是谁？
中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
""",
    },
]

# 定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 可供选择的示例列表
    examples,
    # 用于生成嵌入的嵌入类，用于衡量语义相似性
    embeddings_model,
    # 这是用于存储嵌入并进行相似性搜索的VectorStore类
    Chroma,
    # 这是要生成的示例数量
    k=1
)

# 选择与输入最相似的示例
question = "《大白鲨》的导演是？"
selected_examples = example_selector.select_examples({"question": question})
print(f"与输入最相似的示例：{selected_examples}")

与输入最相似的示例：[{'question': '《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？', 'answer': '\n接下来还需要问什么问题吗？\n追问：《大白鲨》的导演是谁？\n中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。\n'}]


In [12]:
import os
import dotenv
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()

# 定义示例提示词模板
example_prompt = PromptTemplate.from_template(
    template="Input: {input}\nOutput: {output}"
)

# 创建一个示例提示词模板
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "喜欢", "output": "不喜欢"},
    {"input": "爱", "output": "讨厌"},
    {"input": "依恋", "output": "排斥"},  # 新增：情感类
    {"input": "思念", "output": "遗忘"},  # 新增：情感类
    {"input": "爱慕", "output": "憎恶"},  # 新增：情感类
    {"input": "高", "output": "低"},
    {"input": "好", "output": "坏"},
    {"input": "精力充沛", "output": "懒惰"},
    {"input": "成功", "output": "失败"},
    {"input": "强", "output": "弱"},
]

# 定义嵌入模型
embeddings_model = OpenAIEmbeddings(
    model=os.getenv("LLM_EMBEDINGS_MODEL")
)

# 创建语义相似性示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings_model,
    FAISS,
    k=1
)

# 定义小样本提示词模板
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个词组的反义词",
    suffix="Input: {word}\nOutput:",
    input_variables=["word"]
)
input_var = input("请输入一个词：")
response = similar_prompt.invoke({"word": input_var})
print(response.text)

给出每个词组的反义词

Input: 高
Output: 低

Input: 爱
Output:


In [15]:
from langchain_core.prompts import load_prompt
from dotenv import load_dotenv

load_dotenv()

prompt = load_prompt("asset/prompt.yml", encoding="utf-8")
print(prompt.format(name="年轻人", what="恐怖"))

请给年轻人讲一个关于恐怖的故事


In [16]:
from langchain_core.prompts import load_prompt
from dotenv import load_dotenv

load_dotenv()

prompt = load_prompt("asset/prompt.json", encoding="utf-8")
print(prompt.format(name="年轻人", what="恐怖"))

请给年轻人讲一个关于恐怖的故事
